## I. Построение классификаторов изображений

### - Запускаем Graph Lab Create

In [117]:
import graphlab
graphlab.set_runtime_config('GRAPHLAB_DEFAULT_NUM_PYLAMBDA_WORKERS', 4)

### - Загрузка датасета CIFAR-10 

In [118]:
image_train = graphlab.SFrame('image_train_data/')
image_test = graphlab.SFrame('image_test_data/')

### - Показать изображения

In [119]:
graphlab.canvas.set_target('ipynb')
image_train['image'].show()

### - Показать информацию о таблице (структуру)

In [120]:
image_train.head()

id,image,label,deep_features,image_array
24,Height: 32 Width: 32,bird,"[0.242871761322,1.09545373917, 0.0, ...","[73.0, 77.0, 58.0, 71.0,68.0, 50.0, 77.0, 69.0, ..."
33,Height: 32 Width: 32,cat,"[0.525087952614, 0.0,0.0, 0.0, 0.0, 0.0, ...","[7.0, 5.0, 8.0, 7.0, 5.0,8.0, 5.0, 4.0, 6.0, 7.0, ..."
36,Height: 32 Width: 32,cat,"[0.566015958786, 0.0,0.0, 0.0, 0.0, 0.0, ...","[169.0, 122.0, 65.0,131.0, 108.0, 75.0, ..."
70,Height: 32 Width: 32,dog,"[1.12979578972, 0.0, 0.0,0.778194487095, 0.0, ...","[154.0, 179.0, 152.0,159.0, 183.0, 157.0, ..."
90,Height: 32 Width: 32,bird,"[1.71786928177, 0.0, 0.0,0.0, 0.0, 0.0, ...","[216.0, 195.0, 180.0,201.0, 178.0, 160.0, ..."
97,Height: 32 Width: 32,automobile,"[1.57818555832, 0.0, 0.0,0.0, 0.0, 0.0, ...","[33.0, 44.0, 27.0, 29.0,44.0, 31.0, 32.0, 45.0, ..."
107,Height: 32 Width: 32,dog,"[0.0, 0.0,0.220677852631, 0.0, ...","[97.0, 51.0, 31.0, 104.0,58.0, 38.0, 107.0, 61.0, ..."
121,Height: 32 Width: 32,bird,"[0.0, 0.23753464222, 0.0,0.0, 0.0, 0.0, ...","[93.0, 96.0, 88.0, 102.0,106.0, 97.0, 117.0, ..."
136,Height: 32 Width: 32,automobile,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 7.5737862587, 0.0, ...","[35.0, 59.0, 53.0, 36.0,56.0, 56.0, 42.0, 62.0, ..."
138,Height: 32 Width: 32,bird,"[0.658935725689, 0.0,0.0, 0.0, 0.0, 0.0, ...","[205.0, 193.0, 195.0,200.0, 187.0, 193.0, ..."


##  Создаем модель на обучающих данных NNMׂ(Nearest Neighbor Model)

In [121]:
knn_model =graphlab.nearest_neighbors.create(image_train,features=['deep_features'],label='id')

Starting brute force nearest neighbors model training.

# Практическая работа

# 1.	Используя метод .sketch_summary() для таблицы image_train, выве-сти с указанием количеств, каких типов картинок больше всего в таблице, а каких – меньше всего. 

In [122]:
image_train['label'].sketch_summary()


+------------------+-------+----------+
|       item       | value | is exact |
+------------------+-------+----------+
|      Length      |  2005 |   Yes    |
| # Missing Values |   0   |   Yes    |
| # unique values  |   4   |    No    |
+------------------+-------+----------+

Most frequent items:
+-------+------------+-----+-----+------+
| value | automobile | cat | dog | bird |
+-------+------------+-----+-----+------+
| count |    509     | 509 | 509 | 478  |
+-------+------------+-----+-----+------+


# 2.	Выбрать из таблицы image_train только картинки по типу, заданно-му в варианте. Получить ближайших соседей к первой картинке в полученном множестве.

cat and bird

In [123]:
catbird = image_train[(image_train['label']== 'cat')|(image_train['label']== 'bird')]
catbird['image'].show()

In [124]:
graphlab.canvas.set_target('ipynb')
mypicture = catbird[0:1]
mypicture['image'].show()

In [125]:
knn_model.query(mypicture)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 31.2ms       |

| Done         |         | 100         | 327.601ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,24,0.0,1
0,30224,36.5454765642,2
0,34071,37.8011450878,3
0,37742,41.0399299616,4
0,36130,41.5105447214,5


In [126]:
picture_neighbors = get_images_from_ids(knn_model.query(mypicture))

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 15.6ms       |

| Done         |         | 100         | 296.401ms    |

+--------------+---------+-------------+--------------+

In [127]:
picture_neighbors['image'].show()

# 3.	Для первой картинки в выбранном множестве по варианту (см. п. 2) рассчитать среднее расстояние между нею и первыми 5 ближайшими сосе-дями. Расстояние находится в колонке ‘distance’ результирующей таблицы при получении ближайших соседей. 

In [128]:
avr = graphlab.SFrame()
avr=knn_model.query(mypicture)['distance']


Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 15.6ms       |

| Done         |         | 100         | 312ms        |

+--------------+---------+-------------+--------------+

In [129]:
avr

dtype: float
Rows: 5
[0.0, 36.54547656421385, 37.80114508781701, 41.03992996157689, 41.51054472137894]

In [130]:
import numpy

In [131]:
avr.mean()

31.37941926699734